In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from __future__ import print_function, division
%tensorflow_version 1.x
from keras.datasets import mnist, cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import sys
import numpy as np

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100
batch_size = 16

In [ ]:
# Load the dataset
(X_train, _), (_, _) = mnist.load_data()

# Rescale -1 to 1
X_train = X_train / 127.5 - 1.
X_train = np.expand_dims(X_train, axis=3)

#  define Adversarial ground truths
real = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

In [ ]:
def build_generator(latent_dim,img_shape):
  
  g_model = Sequential()
  g_model.add(Dense(256, activation = 'relu', input_dim=latent_dim))
  
  g_model.add(BatchNormalization(momentum=0.8))
  g_model.add(Dense(512, activation = 'relu'))
  
  g_model.add(BatchNormalization(momentum=0.8))
  g_model.add(Dense(1024, activation = 'relu'))
  
  g_model.add(BatchNormalization(momentum=0.8))
  g_model.add(Dense(np.prod(img_shape), activation='tanh'))
  g_model.add(Reshape(img_shape))

  g_model.summary()
  noise = Input(shape=(latent_dim,))
  image = g_model(noise)

  return Model(noise, image)

In [ ]:
def build_discriminator(img_shape):

  d_model = Sequential()

  d_model.add(Flatten(input_shape=img_shape))
  d_model.add(Dense(512, activation = 'relu'))
  d_model.add(Dense(256, activation = 'relu'))
  d_model.add(Dense(1, activation='sigmoid'))
  d_model.summary()

  img = Input(shape=img_shape)
  validity = d_model(img)

  return Model(img, validity)

In [ ]:
# Build and compile the discriminator
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0001, 0.5), metrics=['accuracy'])

# Build the generator
generator = build_generator(latent_dim, img_shape)

# The generator takes noise as input and generates imgs
noise_img = Input(shape=(latent_dim,))
img = generator(noise_img)

# For the combined model we will only train the generator
discriminator.trainable = False

# dicriminator take images and validates the originality
validity = discriminator(img)

# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model(noise_img, validity)
combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0001, 0.5))




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Par

In [ ]:
def savedimages(epoch, latent_dim, generator):
        rows, columns = 4, 4
        noise = np.random.normal(0, 1, (rows * columns, latent_dim))
        genrated_images = generator.predict(noise)

        # Rescale images  in range of 0 to 1
        genrated_images = 0.5 * genrated_images + 0.5

        fig, axis = plt.subplots(rows, columns)
        cnt = 0
        for i in range(rows):
            for j in range(columns):
                axis[i,j].imshow(genrated_images[cnt, :,:,0], cmap='gray')
                axis[i,j].axis('off')
                cnt += 1
        fig.savefig("/content/drive/My Drive/GAN Output/Epoch %d.png" %epoch)
        plt.close()

In [ ]:
epochs = 10000

for epoch in range(epochs):

    # Select a random batch of images
    index = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[index]

    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    # Generate a batch of new images
    gen_imgs = generator.predict(noise)

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(imgs, real)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    # Train the generator (to have the discriminator label samples as valid)
    g_loss = combined.train_on_batch(noise, real)

    # Plot the progress
    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

    # If at save interval => save generated image samples
    if epoch % 300 == 0:
        savedimages(epoch, latent_dim, generator)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.016311, acc.: 46.88%] [G loss: 0.919468]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Streaming output truncated to the last 5000 lines.
5001 [D loss: 0.701160, acc.: 56.25%] [G loss: 1.146278]
5002 [D loss: 0.709247, acc.: 62.50%] [G loss: 1.039990]
5003 [D loss: 0.517510, acc.: 87.50%] [G loss: 1.100006]
5004 [D loss: 0.601808, acc.: 71.88%] [G loss: 1.157770]
5005 [D loss: 0.623405, acc.: 62.50%] [G loss: 1.075721]
5006 [D loss: 0.523062, acc.: 78.12%] [G loss: 0.968798]
5007 [D loss: 0.470589, acc.: 78.12%] [G loss: 1.020323]
5008 [D loss: 0.586547, acc.: 65.62%] [G loss: 1.117140]
5009 [D loss: 0.559826, acc.: 68.75%] [G loss: 1.184091]
5010 [D loss: 0.614904, acc.: 68.75%] [G loss: 1.017824]
5011 [D loss: 0.718246, acc.: 56.25%] [G loss: 0.822491]
5012 [D loss: 0.499559, acc.: 75.00%] [G loss: 1.262762]
5013 [D loss: 0.521431, acc.: 68.75%] [G loss: 1.197683]
5014 [D loss: 0.644984, acc.: 75.00%] [G loss: 0.977510]
5015 [D loss: 0.544899, acc.: 78.12%] [G loss: 0.985044]
5016 [D loss: 0.538189, acc.: 81.25%] [G loss: 0.980731]
5017 [D loss: 0.538580, acc.: 75.00%]